# Installation

Before running this notebook, make sure you have the `iree-turbine` package installed:

In [ ]:
!pip install iree-turbine

# Wave GEMM Kernel Tutorial

This tutorial demonstrates how to write a simple GEMM (General Matrix Multiplication) kernel using the Wave language. We'll walk through the implementation step by step. First, we need to add the necessary imports and define the symbols we will be using in the kernel.

In [66]:
from iree.turbine.kernel._support.indexing import sym
from iree.turbine.kernel._support.dtype import f16, f32
from iree.turbine.kernel.lang.wave_types import *
from iree.turbine.kernel.lang.global_symbols import *
from iree.turbine.kernel.wave.utils.run_utils import set_default_run_config
import iree.turbine.kernel as tkl
import iree.turbine.kernel.wave as tkw
from iree.turbine.kernel.wave.compile import WaveCompileOptions, wave_compile
import torch

# Define symbolic dimensions for our matrices
M = sym.M  # Rows of A and C
N = sym.N  # Rows of B and columns of C
K = sym.K  # Columns of A and B

# Define workgroup tile sizes
BLOCK_M = sym.BLOCK_M
BLOCK_N = sym.BLOCK_N
BLOCK_K = sym.BLOCK_K

# Define the address space for our memory
ADDRESS_SPACE = sym.ADDRESS_SPACE
GLOBAL_ADDRESS_SPACE = sym.GLOBAL_ADDRESS_SPACE

## Mixed Precision GEMM Kernel Implementation

Our GEMM kernel will compute C = A @ B.T, where:
- A is an M×K matrix in f16
- B is a NxK matrix in f16
- C is an M×N matrix in f32

Below, we will define the constraints which specify how we want to distribute the different dimensions of our problem. Then, we will define the kernel.

In [67]:
# Define constraints for the kernel
constraints = [
    tkw.WorkgroupConstraint(M, BLOCK_M, 0),
    tkw.WorkgroupConstraint(N, BLOCK_N, 1),
    tkw.TilingConstraint(K, BLOCK_K),
    tkw.WaveConstraint(M, BLOCK_M / 2),
    tkw.WaveConstraint(N, BLOCK_N / 2),
    tkw.HardwareConstraint(
        threads_per_wave=64,
        waves_per_block=(2, 2, 1),
        mma_type=tkw.MMAType.F32_16x16x16_F16
    )
]

@tkw.wave(constraints)
def gemm(
    a: Memory[M, K, ADDRESS_SPACE, f16],  # Input matrix A
    b: Memory[N, K, ADDRESS_SPACE, f16],  # Input matrix B
    c: Memory[M, N, GLOBAL_ADDRESS_SPACE, f32],  # Output matrix C
):
    # Initialize the accumulator register with zeros
    c_reg = Register[M, N, f32](0.0)

    # Iterate over the K dimension to compute the dot product
    @tkw.iterate(K, init_args=[c_reg])
    def repeat(acc: Register[M, N, f32]) -> Register[M, N, f32]:
        # Load elements from A and B
        a_reg = tkw.read(a)
        b_reg = tkw.read(b)

        # Compute matrix multiplication and accumulate
        acc = tkw.mma(a_reg, b_reg, acc)
        return acc

    # Store the final result to C
    tkw.write(repeat, c)

## Testing the GEMM Kernel

Let's create a test function to verify our GEMM implementation works correctly.

In [70]:
def test_gemm():
    # Create test matrices
    m, n, k = 128, 256, 128  # Small dimensions for testing

    # Initialize input matrices with random values
    torch.manual_seed(0)
    a = torch.randn(m, k, dtype=torch.float16, device="cuda")
    b = torch.randn(n, k, dtype=torch.float16, device="cuda")
    c = torch.zeros(m, n, dtype=torch.float32, device="cuda")

    # Set hyperparameters for compilation
    hyperparams = {
        ADDRESS_SPACE: SHARED_ADDRESS_SPACE,
        BLOCK_M: 64,
        BLOCK_N: 64,
        BLOCK_K: 32,
        M: m,
        N: n,
        K: k,
    }

    # Compile the kernel
    options = WaveCompileOptions(
        subs=hyperparams,
    )
    options = set_default_run_config(options)
    compiled_gemm = wave_compile(options, gemm)

    # Run the GEMM kernel
    compiled_gemm(a, b, c)

    # Verify the result using PyTorch's matmul
    expected = torch.matmul(a, b.t())

    # Check if results are close (accounting for floating-point precision)
    assert torch.allclose(c.to(torch.float16), expected, rtol=1e-2, atol=1e-2), \
        f"GEMM result doesn't match expected output\nMax difference: {(c - expected).abs().max()}"

    print("GEMM test passed!")

# Run the test
test_gemm()

GEMM test passed!


## Explanation of Key Components

1. **Memory Types**:
   - `Memory[M, K, ADDRESS_SPACE, f16]` defines a matrix in memory with dimensions M×K
   - `f16` and `f32` specify the data types (half and single precision)

2. **Wave Dialect Features**:
   - `@tkw.wave()` decorator with constraints defines the kernel's execution parameters
   - `@tkw.iterate` creates a iteration loop over the K dimension
   - `Register` represents values in registers during computation

3. **Key Operations**:
   - `tkw.read` loads values from memory into registers
   - `tkw.mma` performs matrix multiplication and accumulation
   - `tkw.write` writes results back to memory

4. **Constraints**:
   - Workgroup constraints define how the computation is tiled
   - Wave constraints specify how data is distributed across waves
   - Hardware constraints define the execution environment

This implementation demonstrates the basic structure of a wave kernel and how to use the wave language's features for efficient matrix multiplication.